## Homework 10: GPUs

## Due Date: May 04, 2020, 08:00am

#### Firstname Lastname: Chengwei Chen

#### E-mail: cc6576@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (100p)**


Use PyOpenCL to write a program which will be able to run in parallel on GPUs. 


Your program will:

- draw two random vectors $\vec u$ and $\vec v$ from $[0,1]^N$ where $N = 10^7$;


- calculate and output similarity between $\vec u$ and $\vec v$.




The similarity between two vectors $\vec u$ and $\vec v$ is defined here as a `cosine` value of the angle between them $\measuredangle \left( \vec u, \vec v \right)$. That is, the program returns: 

$$\cos \left( \measuredangle \left( \vec u, \vec v \right) \right)$$.


Note that the output is a real value and must belong to $[-1, 1]$.

In [12]:
import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array 
#(a Numpy array plus an OpenCL buffer object)
import numpy as np  # Import Numpy number tools

platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.create_some_context()  # Initialize the Context
queue   = cl.CommandQueue(context)  # Instantiate a Queue

N = 10_000_000
# Create two random pyopencl arrays
u = pycl_array.to_device(queue, np.random.rand(N, 1).astype(np.float32))
v = pycl_array.to_device(queue, np.random.rand(N, 1).astype(np.float32))
# pycl_array.to_device(queue, np.random.rand(10000000).astype(np.float32))

# Create an empty pyopencl destination array
res_c = pycl_array.empty_like(u) 
norm_u = pycl_array.empty_like(u)
norm_v = pycl_array.empty_like(u)

program = cl.Program(context, """
__kernel void cosine_similarity(__global const float *u, __global const float *v, __global float *s, __global float *norm_u,__global float *norm_v )
{
  int i = get_global_id(0);
  s[i] = u[i]*v[i];
  norm_u[i] = u[i]*u[i];
  norm_v[i] = v[i]*v[i];
}""").build()

# Enqueue the program for execution and store the result in c
program.cosine_similarity(queue, u.shape, None, u.data, v.data, res_c.data, norm_u.data, norm_v.data)

s = np.sum(res_c.get())
n_u = (np.sum(norm_u.get()))**0.5
n_v = (np.sum(norm_v.get()))**0.5

print(s / (n_u*n_v))
# Check on CPU with Numpy:
# print(np.dot(np.transpose(u), v) / (np.linalg.norm(u)*np.linalg.norm(v))) 


0.7499447937570847
